In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import codecs
from datetime import datetime

In [2]:
path = Path('.').absolute()
path

WindowsPath('C:/Users/Eligio/Documents/NPEC/NEAT/02.DataCollection')

In [3]:
# files
flist = ['1. Collected_data-China_testc1_modisa.csv', 
         '2. Collected_data-Korea_red_tite_south_sea(2018-2019)_UTC.csv',
         '2. Collected_data-Korea_wave_glider_in_situ_data.csv',
         '3. Collected_data-Russia_VKachur_chl_a_new.csv',
         '3. Collected_data-Russia_VKachur_chl_OC2.csv',
         '3. Collected_data-Russia_VKachur_Rrs.csv', 'Toyama_data2003-2021.xls']

In [4]:
def china(file: str):
    dataset = pd.read_csv(path.joinpath(file))
    dataset.rename(columns={col:col.lower() 
                            for col in dataset.columns}, 
                   inplace=True)
    dataset.insert(1, 'country', 'China')
    return dataset.set_index(index)

In [5]:
def korea(file: str):
    ds = pd.read_csv(path.joinpath(file))
    if 'red_tite' in file:
        ds.rename(columns={'time(UTC)':'time'}, inplace=True)
        ds.insert(1, 'country', 'Korea')
        return ds.set_index(index)
    
    if 'wave_glider' in file:
        dates, times = [], []
        append = dates.append
        appent = times.append
        
        drop_cols = ['Year', 'Month', 'Day', 'Hour', 'Minute'] + [
            col for col in ds.columns if 'Unnamed' in col
        ]
        for y, m, d, h, mn in zip(ds.loc[: ,'Year'], 
                                  ds.loc[: ,'Month'],
                                  ds.loc[: ,'Day'],
                                  ds.loc[: ,'Hour'], 
                                  ds.loc[: ,'Minute']):
            # date = pd.to_datetime(f'{y}-{d:02}-{m:02} {h:02}:{mn:02}')
            append(f'{m:g}/{d:g}/{y}')
            appent(f'{h:g}:{mn:02}')
        
        sta = [f'WG{i+1}' for i in range(ds.shape[0])]
        ds.insert(0, 'station', sta)
        ds.insert(1, 'country', 'Korea')
        ds.insert(2, 'date', dates)
        ds.insert(3, 'time', times)
        ds.drop(columns=drop_cols, inplace=True)
        
        ds.rename(columns={'Longitude':'lon', 
                           'Latitude': 'lat', 
                           'Chlorophyll-a': 'chla'}, inplace=True)
        return ds.set_index(index)

In [6]:
def japan(file: str):
    ds = pd.read_excel(path.joinpath(file), sheet_name='Sheet2', parse_dates=False)
    cols = {col:col.replace('Rrs', 'Rrs_') for col in ds.columns}
    ds.sort_values('date', axis=0, ascending=True, inplace=True)
    ds.sort_values('date', axis=0, ascending=True, inplace=True)
    ds.dropna(axis=0, subset=['date', 'time'], inplace=True)
    
    ds['date'] = [fmt_date(date_str=d, sep='-') for d in ds['date']]
    ds['time'] = [fmt_time(time_str=t) for t in ds['time']]
    ds.rename(columns=cols, inplace=True)
    ds.replace(np.nan, '-999', inplace=True)
    return ds.set_index(index)

In [7]:
def fmt_time(time_str: str, sep: str = ':'):
    # print(time_str, type(time_str))
    if type(time_str) == str:
        h, m, s = time_str.split(sep)
    else:
        h, m = time_str.strftime(f'%H{sep}%M').split(sep)
    return f'{int(h)}{sep}{int(m):02}'

In [8]:
def fmt_date(date_str: str, sep: str = '/'):
    # print(date_str, type(date_str))
    if type(date_str) == str:
        m, d, y = date_str.split(sep)
    else:
        m, d, y = date_str.strftime(f'%m{sep}%d{sep}%Y').split(sep)
    return f'{int(m)}/{int(d)}/{int(y)}'

In [9]:
def russia(file: str):
    dataset = pd.read_csv(path.joinpath(file))
    if 'Rrs' in file:
        dataset['date'] = [fmt_date(date_str=d) for d in dataset['date']]     
    dataset.insert(1, 'country', 'Russia')
    return dataset.set_index(index)

In [10]:
def merge_pd(left=None, right=None, top=None, bottom=None):
    if left is None:
        return top.append(other=bottom)
    if top is None:
        return left.merge(right, left_index=True, right_index=True, how='left')

In [11]:
# !pip install openpyxl
# !python -m pip install --upgrade pip

In [12]:
df = None
index = ['country', 'date', 'time', 'lat', 'lon', 'station']
for i, f in enumerate(flist):
        
    if df is None:
        df = china(file=f)
        continue
        
    if i < 5:
        if int(f[0]) == 2:
            dfi = korea(file=f)
        if int(f[0]) == 3:
            dfi = russia(file=f)
        df = merge_pd(top=df, bottom=dfi)
        
    if i == 5:
        dfi = russia(file=f)
        df = merge_pd(left=df, right=dfi)
        
    if not f[0].isdigit():
        dfi = japan(file=f)
        df = merge_pd(top=df, bottom=dfi)
        cols = [col for col in df.columns if 'Rrs_' not in col] + sorted(
            [col for col in df.columns if 'Rrs_' in col]
        )
        df = df.loc[:, cols]
    print(f'{f[0]}: {df.shape}')
df.replace(np.nan, '-999', inplace=True)

2: (57, 1)
2: (1426, 1)
3: (1544, 1)
3: (1813, 2)
3: (1813, 293)
T: (2858, 301)


In [13]:
dstr = datetime.today().strftime('%Y%m%d')
f = f'insitu_data_collection_{dstr}.csv'
df.to_csv(f, float_format='%.8f')

In [14]:
def fmt_output(iterator: iter):
    result = []
    append = result.append
    for ln, hd in iterator:
        if (hd in index) or (ln in ('-999', 999)):
            append(ln)
            continue
        try:
            append(f'{float(ln):.6f}')
        except ValueError:
            append('-999')
    return result

In [15]:
# split = '\r\n'
# dstr = datetime.today().strftime('%Y%m%d')
# sds = df.reset_index().to_csv(index=False).split(split)
# f = f'insitu_data_collection_{dstr}.csv'
# with codecs.open(f, mode='w', encoding='utf-8') as txt:
#     meta = sds.pop(0)
#     txt.write(f'{meta}\n')
#     meta = meta.split(split)[0].split(',')
    
#     for i, line in enumerate(sds):
#         li = line.split(split)[0].split(',')
#         ret = fmt_output(iterator=zip(li, meta))
#         str_fmt = ','.join(ret)
#         if i == 1815:
#             print(str_fmt)
#         txt.write(f'{str_fmt}\n')

In [16]:
# # 1: (40, 1)
# # 2: (57, 1)
# # 2: (1426, 1)
# # 3: (1544, 1)
# # 3: (1813, 2)
# # 3: (1813, 293)
# # T: (1813, 293)
# # ds1 = russia(file=flist[4]).reset_index()
# # ds1 = ds1.set_index(['station', 'country'])
# # ds2 = russia(file=flist[5]).reset_index()
# # ds2 = ds2.set_index(['station', 'country'])
# ds1 = russia(file=flist[4])
# ds2 = russia(file=flist[5])

In [17]:
# # for i in range(ds1.shape[0]):
# #     idx = ds2.isin(ds1.loc[i, index].to_list())
# #     break
# # idx, ds2, ds1
# out = ds1.merge(ds2, left_index=True, right_index=True, how='left')
# out
# df.merge(dfi, left_index=True, right_index=True, how='left')
idx = df.reset_index()['country'].isin(['Japan'])
df.reset_index().loc[idx, :].to_csv('text.csv')